In [1]:
import re
import pandas as pd
import numpy as np

In [3]:
cpi = pd.read_csv('../../data/03_외부데이터/CPI(2019년~2020년).csv', encoding = 'CP949')

# Category work

In [4]:
raw_cat = list(cpi.지출목적별)

In [5]:
usable_cats = raw_cat[:292] # 06.보건부터 버림

In [6]:
big_cats = list(filter(lambda x: x[0] in map(str, range(0, 10)), map(str, usable_cats)))

In [7]:
small_cats = list(filter(lambda x: re.findall(r'\d\d\.', x), big_cats))

In [8]:
small_idx = list(map(usable_cats.index, small_cats))
small_idx

[2, 137, 149, 157, 161, 194, 201, 205, 209, 217, 229, 239, 243, 262, 272, 277]

In [9]:
hier = dict()

In [10]:
unused = ['비주류음료', '주류', '담배', '주택임차료', '주거시설유지·보수', '수도및주거관련서비스', '전기·가스및기타연료']

In [11]:
for i in range(len(small_idx) - 2 + 1):
    start, end = small_idx[i:i+2]
    
    key = usable_cats[start][5:].replace(' ', '')
    if key not in unused:
        hier[key] = list(map(lambda x: x.replace(' ', ''), usable_cats[start+1:end]))

In [12]:
hier.keys()

dict_keys(['식료품', '의류', '신발', '가구·가사비품및카페트', '가정용섬유제품', '가정용기기', '주방용품및가정용품', '가정·정원용공구및장비'])

In [13]:
hier

{'식료품': ['빵및곡물',
  '쌀',
  '현미',
  '찹쌀',
  '보리쌀',
  '콩',
  '땅콩',
  '혼식곡',
  '밀가루',
  '국수',
  '라면',
  '당면',
  '두부',
  '시리얼',
  '부침가루',
  '케이크',
  '빵',
  '떡',
  '파스타면',
  '육류',
  '국산쇠고기',
  '수입쇠고기',
  '돼지고기',
  '닭고기',
  '소시지',
  '햄및베이컨',
  '어류및수산',
  '갈치',
  '명태',
  '조기',
  '고등어',
  '오징어',
  '게',
  '굴',
  '조개',
  '전복',
  '마른멸치',
  '마른오징어',
  '낙지',
  '오징어채',
  '북어채',
  '어묵',
  '맛살',
  '생선통조림',
  '젓갈',
  '우유·치즈및계란',
  '우유',
  '분유',
  '치즈',
  '발효유',
  '달걀',
  '식용유지',
  '참기름',
  '식용유',
  '과일',
  '사과',
  '배',
  '복숭아',
  '포도',
  '밤',
  '감',
  '귤',
  '오렌지',
  '참외',
  '수박',
  '딸기',
  '바나나',
  '키위',
  '블루베리',
  '아몬드',
  '과일가공품',
  '채소및해조',
  '배추',
  '상추',
  '시금치',
  '양배추',
  '미나리',
  '깻잎',
  '부추',
  '무',
  '열무',
  '당근',
  '감자',
  '고구마',
  '도라지',
  '콩나물',
  '버섯',
  '오이',
  '풋고추',
  '호박',
  '가지',
  '토마토',
  '파',
  '양파',
  '마늘',
  '브로콜리',
  '고사리',
  '파프리카',
  '단무지',
  '김',
  '맛김',
  '미역',
  '과자·빙과류및당류',
  '초콜릿',
  '사탕',
  '껌',
  '아이스크림',
  '비스킷',
  '스낵과자',
  '파이',
  '설탕',
  '잼',
  '꿀',
  '물엿',
  '기타식

# See Original...

In [62]:
train = pd.read_excel(r'../../data/01_제공데이터/2020 빅콘테스트 데이터분석분야-챔피언리그_2019년 실적데이터.xlsx',
                   header=1,
                   names=['방송일시', '노출(분)', '마더코드', '상품코드',
                          '상품명', '상품군', '판매단가', '취급액'],
                   thousands=',',
                   )

In [63]:
train = train[:-9]

In [64]:
prod_names = (train[train.상품군 != '무형'].상품명 + ' ' + train[train.상품군 != '무형'].상품군).values# len(prod_names) > 1692 (무형 빼고)

In [65]:
from konlpy.tag import Komoran
komoran = Komoran(userdic='../../data/04_추가데이터/user_dict_0913.txt')
print(komoran.nouns("KOMORAN은 한국어 형태소 분석기입니다."))

naive_tokens = list(map(komoran.pos, prod_names))

['한국어', '형태소', '분석기']


In [66]:
def morph_filter(lst):
    return list(map(lambda x: x[0], filter(lambda x: (x[1] in usable_morphs), lst)))

usable_morphs = ['NNG', 'NNP', 'NP', 'NR'] #+ ['SL', 'SN']
naive_tokens = list(map(morph_filter, naive_tokens))

In [67]:
exclude_words = ['종', '인용']
def exclude(lst):
    return list(filter(lambda x: x not in exclude_words, lst))
processed_tokens = list(map(exclude, naive_tokens))

In [68]:
processed_tokens

[['테이트', '남성', '셀린니트', '의류'],
 ['테이트', '여성', '셀린니트', '의류'],
 ['테이트', '남성', '셀린니트', '의류'],
 ['테이트', '여성', '셀린니트', '의류'],
 ['테이트', '남성', '셀린니트', '의류'],
 ['테이트', '여성', '셀린니트', '의류'],
 ['오모떼', '레이스', '파운데이션', '브라', '속옷'],
 ['오모떼', '레이스', '파운데이션', '브라', '속옷'],
 ['오모떼', '레이스', '파운데이션', '브라', '속옷'],
 ['CERINI', 'PAT', '남성', '소프트', '기모', '릴렉스', '팬츠', '의류'],
 ['CERINI', 'PAT', '남성', '소프트', '기모', '릴렉스', '팬츠', '의류'],
 ['CERINI', 'PAT', '남성', '소프트', '기모', '릴렉스', '팬츠', '의류'],
 ['보코', '리버시블', '무스탕', '의류'],
 ['보코', '리버시블', '무스탕', '의류'],
 ['보코', '리버시블', '무스탕', '의류'],
 ['CERINI', 'PAT', '남성', '풀', '패키지', '기모', '니트', '의류'],
 ['CERINI', 'PAT', '남성', '풀', '패키지', '기모', '니트', '의류'],
 ['CERINI', 'PAT', '남성', '풀', '패키지', '기모', '니트', '의류'],
 ['크로커다일', '편안', '코튼', '브라팬티', '세트', '속옷'],
 ['크로커다일', '편안', '코튼', '브라팬티', '세트', '속옷'],
 ['에코라믹', '통주', '무', '스톤', '냄비', '세트', '주방'],
 ['에코라믹', '통주', '무', '스톤', '냄비', '세트', '주방'],
 ['에코라믹', '통주', '무', '스톤', '냄비', '세트', '주방'],
 ['오모떼', '360', '텐션', '업', '레이스', '패키지', '시즌', '

In [22]:
train.상품군.unique()

array(['의류', '속옷', '주방', '농수축', '이미용', '가전', '생활용품', '건강기능', '잡화', '무형',
       '가구', '침구'], dtype=object)

In [23]:
hier.keys()

dict_keys(['식료품', '의류', '신발', '가구·가사비품및카페트', '가정용섬유제품', '가정용기기', '주방용품및가정용품', '가정·정원용공구및장비'])

In [24]:
unique_tokens = list(map(' '.join, processed_tokens))

In [25]:
unique_tokens = np.unique(unique_tokens, axis=0)

In [29]:
unique_tokens = list(map(lambda x: x.split(' '), unique_tokens))

In [30]:
unique_tokens

[['1+1', '보국', '미니', '히터', '화이트', '레드', 'PR', '생활용품'],
 ['1+1', '보국', '미니', '히터', '화이트', '화이트', '생활용품'],
 ['3D', '매', '냉수마찰', '물', '매트', '생활용품'],
 ['94', '웰스', '황사', '방역', '마스크', '생활용품'],
 ['CERINI', 'PAT', '남성', '기모', '릴렉스', '팬츠', '의류'],
 ['CERINI', 'PAT', '남성', '기모', '집업', '니트', '의류'],
 ['CERINI', 'PAT', '남성', '니트', '스웨터', '의류'],
 ['CERINI', 'PAT', '남성', '소프트', '기모', '릴렉스', '팬츠', '의류'],
 ['CERINI', 'PAT', '남성', '스프링', '릴렉스', '팬츠', '의류'],
 ['CERINI', 'PAT', '남성', '썸머', '쿠', '카라', '티셔츠', '의류'],
 ['CERINI', 'PAT', '남성', '어반', '릴렉스', '팬츠', '의류'],
 ['CERINI', 'PAT', '남성', '어반', '카라', '티셔츠', '8월', '의류'],
 ['CERINI', 'PAT', '남성', '어반', '카라', '티셔츠', '의류'],
 ['CERINI', 'PAT', '남성', '에어', '카', '티셔츠', '의류'],
 ['CERINI', 'PAT', '남성', '에어', '카라', '티셔츠', '의류'],
 ['CERINI', 'PAT', '남성', '여름', '에어', '윈', '디', '릴렉스', '팬츠', '의류'],
 ['CERINI', 'PAT', '남성', '집업', '니트', '의류'],
 ['CERINI', 'PAT', '남성', '캐주얼', '셔츠', '의류'],
 ['CERINI', 'PAT', '남성', '풀', '패키지', '기모', '니트', '의류'],
 ['F/W', '남성', '리버시블', '자켓',

## Sliding Window 사용해보기..

는 fail

In [21]:
from itertools import *
import itertools

In [22]:
def sliding_window(iterable, n=2):
    iterables = itertools.tee(iterable, n)
    
    for iterable, num_skipped in zip(iterables, itertools.count()):
        for _ in range(num_skipped):
            next(iterable, None)
    
    return zip(*iterables)

In [24]:
for vals in sliding_window(unique_tokens[1223], 2):
    print(''.join((chain(vals))))

코치
치 
 플
플라
라이
이트
트백
백 
 잡
잡화


In [25]:
for lst in hier.values():
    for vals in sliding_window(unique_tokens[1223], 2):
        print(''.join((chain(vals))) in lst)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


In [26]:
unique_tokens[1267]

'크리스티나앤코 피치기모 팬츠 의류'

In [27]:
list(filter(lambda x: x[-4:] == '생활용품', unique_tokens))

['1+1 보국 미니 히터 화이트 레드 PR 생활용품',
 '1+1 보국 미니 히터 화이트 화이트 생활용품',
 '3D 매 냉수마찰 물 매트 생활용품',
 '94 웰스 황사 방역 마스크 생활용품',
 'LG생활건강 건조기 시트 퍼퓸 드 몽드 생활용품',
 'LG생활건강 샤프 란 아우라 농축 섬유유연제 생활용품',
 'LG생활건강 테크 한장 빨래 화이트 업 박스 생활용품',
 'LG생활건강 히말라야 핑크솔트 화이트 라벨 치약 세트 생활용품',
 '거미 손 차량 공기청정기 생활용품',
 '국내 제작 김병지 밀도 풀 확장 코일 카매트 생활용품',
 '국내 제조 따 스림 실크 단열 벽지 미터 30 미터 생활용품',
 '국내 제조 따 스림 실크 단열 벽지 미터 생활용품',
 '국내 제조 벨라홈 논슬립 옷걸이 세트 생활용품',
 '굿프렌드 굿 터치 클리너 1+1 생활용품',
 '굿프렌드 굿 터치 클리너 생활용품',
 '근육통 완화 동전 패치 생활용품',
 '기본 구성 바로톡 싱크대 거름통 생활용품',
 '김병만 정글 피싱 낚시 세트 생활용품',
 '김병지 거미 손 풀 확장 일 체형 코일 카매트 생활용품',
 '까 사 마루 행거 단 생활용품',
 '까 사마 멀티 행거 세트 생활용품',
 '까 사마 접이식 벤치 의자 인 생활용품',
 '까 사마 접이식 의자 생활용품',
 '까 사마 접이식 테이블 세트 1+1 세트 생활용품',
 '까 사마 접이식 테이블 세트 생활용품',
 '노비타 스마트 비데 생활용품',
 '노송 가구 제이드 비취 교자상 생활용품',
 '노송 가구 제이드 비취 꽃 상감 교자상 생활용품',
 '대형 한솔 뉴청맥 온열 카페트매트 생활용품',
 '더블 더블 일월 순면 온열 매트 생활용품',
 '더블 사이즈 일월 순면 온열 매트 생활용품',
 '더블 싱글 일월 순면 온열 매트 생활용품',
 '데시벨 블루투스 이어폰 1000 생활용품',
 '데시벨 블루투스 이어폰 A1 생활용품',
 '도루 코 페이스 면도기 세트 1+1 세트 생활용품',
 '도루 코 페이스 면도기 세

## Using Set Intersection

In [53]:
for k in hier:
    hier[k].append(k)

In [104]:
cnt = 0
not_filt = 0
for product in unique_tokens:
    
    product = set(product)
    filt = False
    for k, v in hier.items():
        ref = set(v)
        intersect = product.intersection(ref)
        
        if intersect:
            filt = True
            cnt += 1
            break
            
        elif '건강기능' in product:
            filt = True
            cnt += 1
            break
            
        elif '주방' in product:
            filt = True
            cnt += 1
            break
            
        elif '속옷' in product:
            filt = True
            cnt += 1
            break
            
    if not filt:
        
        if any(list(filter(lambda x: )))
        
        for k, v in extra_hier.items():
            ref = set(v)
            intersect = product.intersection(ref)
            if intersect:
                filt = True
                cnt += 1
                break
                
        if not filt:
            print(product)
            not_filt += 1

{'보국', '화이트', '1+1', 'PR', '생활용품', '레드', '히터', '미니'}
{'보국', '화이트', '1+1', '생활용품', '히터', '미니'}
{'냉수마찰', '매트', '생활용품', '매', '3D', '물'}
{'94', '생활용품', '마스크', '방역', '황사', '웰스'}
{'LA갈비', '팩', '농수축'}
{'LG', 'TV', '울트라HD', '가전'}
{'시트', '몽드', '건조기', 'LG생활건강', '생활용품', '퍼퓸', '드'}
{'란', '샤프', '아우라', '농축', '섬유유연제', 'LG생활건강', '생활용품'}
{'한장', '화이트', '테크', 'LG생활건강', '박스', '생활용품', '빨래', '업'}
{'세트', '화이트', 'LG생활건강', '생활용품', '라벨', '히말라야', '핑크솔트', '치약'}
{'가격', '팩', '갈비살', '인하', '전철우', '농수축', '구이'}
{'탕요일', '농수축', '강레오'}
{'팩', '강원도', '시래기', '15', '농수축', '양구'}
{'플러스', '이미용', '고데롤'}
{'봉', '곡물', '농수축'}
{'봉', '곡물', '오리지널', '농수축'}
{'곱창전골', '팩', '농수축'}
{'구찌', '선글라스', '잡화', '오버', '더블'}
{'씨마', '구찌', '플랩', '잡화', '장지갑', '단', '마이크로'}
{'팩', '참조기', '산', '농수축', '국내', '12'}
{'손질', '가자미', '농수축', '미', '국내'}
{'팩', '갑오징어', '손질', '농수축', '국내'}
{'팩', '손질', '농수축', '국내', '꼬막'}
{'손질', '농수축', '미', '국내', '병어'}
{'통오징어', '손질', '농수축', '미', '국내'}
{'팩', '통오징어', '손질', '농수축', '국내'}
{'팩', '햇', '갑오징어', '손질', '농수축', '국내'}
{'생물', '신안', '농수축', '

In [105]:
cnt, not_filt

(1113, 383)

In [102]:
len(unique_tokens)

1496

In [98]:
hier

{'식료품': ['빵및곡물',
  '쌀',
  '현미',
  '찹쌀',
  '보리쌀',
  '콩',
  '땅콩',
  '혼식곡',
  '밀가루',
  '국수',
  '라면',
  '당면',
  '두부',
  '시리얼',
  '부침가루',
  '케이크',
  '빵',
  '떡',
  '파스타면',
  '육류',
  '국산쇠고기',
  '수입쇠고기',
  '돼지고기',
  '닭고기',
  '소시지',
  '햄및베이컨',
  '어류및수산',
  '갈치',
  '명태',
  '조기',
  '고등어',
  '오징어',
  '게',
  '굴',
  '조개',
  '전복',
  '마른멸치',
  '마른오징어',
  '낙지',
  '오징어채',
  '북어채',
  '어묵',
  '맛살',
  '생선통조림',
  '젓갈',
  '우유·치즈및계란',
  '우유',
  '분유',
  '치즈',
  '발효유',
  '달걀',
  '식용유지',
  '참기름',
  '식용유',
  '과일',
  '사과',
  '배',
  '복숭아',
  '포도',
  '밤',
  '감',
  '귤',
  '오렌지',
  '참외',
  '수박',
  '딸기',
  '바나나',
  '키위',
  '블루베리',
  '아몬드',
  '과일가공품',
  '채소및해조',
  '배추',
  '상추',
  '시금치',
  '양배추',
  '미나리',
  '깻잎',
  '부추',
  '무',
  '열무',
  '당근',
  '감자',
  '고구마',
  '도라지',
  '콩나물',
  '버섯',
  '오이',
  '풋고추',
  '호박',
  '가지',
  '토마토',
  '파',
  '양파',
  '마늘',
  '브로콜리',
  '고사리',
  '파프리카',
  '단무지',
  '김',
  '맛김',
  '미역',
  '과자·빙과류및당류',
  '초콜릿',
  '사탕',
  '껌',
  '아이스크림',
  '비스킷',
  '스낵과자',
  '파이',
  '설탕',
  '잼',
  '꿀',
  '물엿',
  '기타식

## 예외처리 수작업

In [57]:
extra_hier = {k: [] for k in hier.keys()}

In [79]:
extra_hier

{'식료품': [],
 '의류': [],
 '신발': [],
 '가구·가사비품및카페트': [],
 '가정용섬유제품': [],
 '가정용기기': [],
 '주방용품및가정용품': [],
 '가정·정원용공구및장비': [],
 '기타의류및의류장신구': ['백',
  '호보백',
  '투웨이백',
  '크로스백',
  '숄더백',
  '숄더',
  '토트백',
  '버킷백',
  '미란다백',
  '크로스바디백',
  '플라이트백',
  '사첼백',
  '리스틀릿',
  '다이아',
  '워치',
  '팔찌',
  '주얼리',
  '골드',
  '가방',
  '백팩',
  '머플러',
  '스톨']}

In [76]:
extra_hier['기타의류및의류장신구'] = [
    '백',
    '호보백',
    '투웨이백',
    '크로스백',
    '숄더백',
    '숄더',
    '토트백',
    '버킷백',
    '미란다백',
    '크로스바디백',
    '플라이트백',
    '사첼백',
    '리스틀릿',
    '다이아',
    '워치',
    '팔찌',
    '주얼리',
    '골드',
    '가방',
    '백팩',
    '머플러',
    '스톨',
]

In [81]:
extra_hier['신발'] = [
    '스니커즈',
    '워킹화',
    '트레킹화',
    '러닝화',
    '트레일화',
    '슬립온',
    '샌들',
]

In [55]:
hier.keys()

dict_keys(['식료품', '의류', '신발', '가구·가사비품및카페트', '가정용섬유제품', '가정용기기', '주방용품및가정용품', '가정·정원용공구및장비'])

In [56]:
hier

{'식료품': ['빵및곡물',
  '쌀',
  '현미',
  '찹쌀',
  '보리쌀',
  '콩',
  '땅콩',
  '혼식곡',
  '밀가루',
  '국수',
  '라면',
  '당면',
  '두부',
  '시리얼',
  '부침가루',
  '케이크',
  '빵',
  '떡',
  '파스타면',
  '육류',
  '국산쇠고기',
  '수입쇠고기',
  '돼지고기',
  '닭고기',
  '소시지',
  '햄및베이컨',
  '어류및수산',
  '갈치',
  '명태',
  '조기',
  '고등어',
  '오징어',
  '게',
  '굴',
  '조개',
  '전복',
  '마른멸치',
  '마른오징어',
  '낙지',
  '오징어채',
  '북어채',
  '어묵',
  '맛살',
  '생선통조림',
  '젓갈',
  '우유·치즈및계란',
  '우유',
  '분유',
  '치즈',
  '발효유',
  '달걀',
  '식용유지',
  '참기름',
  '식용유',
  '과일',
  '사과',
  '배',
  '복숭아',
  '포도',
  '밤',
  '감',
  '귤',
  '오렌지',
  '참외',
  '수박',
  '딸기',
  '바나나',
  '키위',
  '블루베리',
  '아몬드',
  '과일가공품',
  '채소및해조',
  '배추',
  '상추',
  '시금치',
  '양배추',
  '미나리',
  '깻잎',
  '부추',
  '무',
  '열무',
  '당근',
  '감자',
  '고구마',
  '도라지',
  '콩나물',
  '버섯',
  '오이',
  '풋고추',
  '호박',
  '가지',
  '토마토',
  '파',
  '양파',
  '마늘',
  '브로콜리',
  '고사리',
  '파프리카',
  '단무지',
  '김',
  '맛김',
  '미역',
  '과자·빙과류및당류',
  '초콜릿',
  '사탕',
  '껌',
  '아이스크림',
  '비스킷',
  '스낵과자',
  '파이',
  '설탕',
  '잼',
  '꿀',
  '물엿',
  '기타식

In [71]:
len(unique_tokens)

1496

In [80]:
1496 - 728

768

In [89]:
cnt

1134